In [ ]:
!pip install numpy matplotlib seaborn

In [3]:
import numpy as np

class PRINCESBoxAnalysis:
    def __init__(self):
        # PRINCE Cipher S-Box
        self.sbox = [
            0xb, 0xf, 0x3, 0x2,
            0xa, 0xc, 0x9, 0x1,
            0x6, 0x7, 0x8, 0x0,
            0xe, 0x5, 0xd, 0x4
        ]

        # Create mappings
        self.sbox_dict = {i: self.sbox[i] for i in range(len(self.sbox))}
        self.inv_sbox_dict = {self.sbox[i]: i for i in range(len(self.sbox))}

    def sbox_transform(self, x):
        """Apply S-box transformation"""
        return self.sbox_dict[x & 0xF]

    def compute_ddt(self):
        """Compute Differential Distribution Table"""
        ddt = np.zeros((16, 16), dtype=int)

        for x1 in range(16):
            for x2 in range(16):
                dx = x1 ^ x2
                dy = self.sbox_transform(x1) ^ self.sbox_transform(x2)
                ddt[dx][dy] += 1

        return ddt



    def analyze_differential_properties(self, ddt):
        """Analyze differential characteristics"""
        # Exclude trivial (0,0) pair
        ddt_without_zero = ddt[1:, :]

        # Find max differential probability
        max_diff_prob = np.max(ddt_without_zero)

        return max_diff_prob

    def compute_differential_branch_number(self):
        """
        Compute the differential branch number of the S-box

        The differential branch number is the minimum Hamming weight
        of the differences of input and output for non-zero input differences.
        """
        min_branch_number = float('inf')

        for dx in range(1, 16):  # Non-zero input differences
            for x1 in range(16):
                x2 = x1 ^ dx
                dy = self.sbox_transform(x1) ^ self.sbox_transform(x2)

                # Compute branch number for this pair
                # Branch number = weight of input difference + weight of output difference
                input_weight = bin(dx).count('1')
                output_weight = bin(dy).count('1')
                branch_number = input_weight + output_weight

                min_branch_number = min(min_branch_number, branch_number)

        return min_branch_number

    def print_ddt(self, ddt):
        """Print Differential Distribution Table"""
        print("\nDifferential Distribution Table (DDT):")
        print("  In\\Out", end=" ")
        for j in range(16):
            print(f"{j:4}", end=" ")
        print()

        for i in range(16):
            print(f"{i:6}", end=" ")
            for j in range(16):
                print(f"{ddt[i][j]:4}", end=" ")
            print()



    def full_analysis(self):
        """Perform comprehensive S-box analysis"""
        # Compute DDT and LAT
        ddt = self.compute_ddt()


        # Differential Analysis
        max_diff_prob = self.analyze_differential_properties(ddt)

        # Differential Branch Number
        diff_branch_number = self.compute_differential_branch_number()

        # Print Results
        print(f"1. Differential Uniformity: {max_diff_prob}/16")
        print(f"2. Differential Branch Number: {diff_branch_number}")

        # Print tables
        self.print_ddt(ddt)


        return ddt

# Run the analysis
analysis = PRINCESBoxAnalysis()
ddt = analysis.full_analysis()

1. Differential Uniformity: 4/16
2. Differential Branch Number: 2

Differential Distribution Table (DDT):
  In\Out    0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15 
     0   16    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
     1    0    4    0    0    2    0    2    0    4    2    0    2    0    0    0    0 
     2    0    2    0    4    0    0    0    2    2    0    0    0    0    4    2    0 
     3    0    0    0    0    0    2    2    0    2    2    2    2    2    0    0    2 
     4    0    2    2    4    2    2    0    0    2    0    2    0    0    0    0    0 
     5    0    0    2    2    0    2    0    2    0    2    0    2    2    2    0    0 
     6    0    0    2    2    0    2    2    0    0    2    0    2    0    0    4    0 
     7    0    0    2    0    0    0    2    0    2    0    4    0    0    2    2    2 
     8    0    0    2    0    4    2    0    0    2    2    0    2    0    2    0    0 
     9    0 

In [5]:
import numpy as np

# Define the S-box
sbox = [
    0xb, 0xf, 0x3, 0x2,
    0xa, 0xc, 0x9, 0x1,
    0x6, 0x7, 0x8, 0x0,
    0xe, 0x5, 0xd, 0x4
]

# Function to calculate parity
def parity(x):
    return bin(x).count('1') % 2

# Initialize the LAT
lat = np.zeros((16, 16), dtype=int)

# Fill the LAT
for input_mask in range(16):
    for output_mask in range(16):
        count = 0
        for x in range(16):
            input_parity = parity(x & input_mask)
            output_parity = parity(sbox[x] & output_mask)
            if input_parity == output_parity:
                count += 1
        lat[input_mask][output_mask] = count - 8

# Display the LAT
print("Linear Approximation Table (LAT):")
print(lat)




Linear Approximation Table (LAT):
[[ 8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 -2 -2  2 -2  0  4 -4  0 -2  2 -2 -2  0  0]
 [ 0  0 -4  0 -4  0  0  0 -2 -2  2 -2 -2  2  2  2]
 [ 0  4 -2 -2  2 -2  0  0  2  2  4  0  0  0  2 -2]
 [ 0  0 -4  4  2  2  2  2  2  2 -2 -2  0  0  0  0]
 [ 0  0  2  2  0 -4 -2  2  2 -2  0 -4 -2 -2  0  0]
 [ 0 -4  0  0 -2 -2  2 -2  0  4  0  0 -2 -2  2 -2]
 [ 0  0  2 -2  0  0 -2  2  0  4 -2 -2  0  4  2  2]
 [ 0 -2 -2  0  4 -2 -2 -4 -2  0  0 -2  2  0  0  2]
 [ 0 -2  0  2  2  0 -2  0  2  0  2  4 -4  2  0  2]
 [ 0  2  2  4  0  2 -2  0 -4  2  2  0  0 -2  2  0]
 [ 0 -2  0 -2  2  4 -2  0  0 -2  0 -2 -2  0  2 -4]
 [ 0 -2 -2  0 -2  0 -4  2  0  2  2  0  2  0 -4 -2]
 [ 0 -2  0  2  0 -2  0  2  0 -2  0  2  4  2  4 -2]
 [ 0 -2  2  0  2  0  4  2 -2  0  4 -2  0  2 -2  0]
 [ 0  2  0  2  0 -2  0 -2 -2  0 -2  0 -2  4 -2 -4]]
